In [1]:
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from wordcloud import WordCloud, STOPWORDS
%matplotlib inline

# Data Read in and Organization

In [2]:
#get current work directory
os.getcwd()

'/Users/Sue/Documents/study2020/springboard/capstone_project2_movie'

In [3]:
# read in data as df
file = '/Users/Sue/Documents/study2020/springboard/capstone_project2_movie/Dataset/tmdb-box-office-prediction/train.csv'
file_test = '/Users/Sue/Documents/study2020/springboard/capstone_project2_movie/Dataset/tmdb-box-office-prediction/test.csv'
data = pd.read_csv(file)
df = pd.DataFrame(data)
print(df.head(5))

   id                              belongs_to_collection    budget  \
0   1  [{'id': 313576, 'name': 'Hot Tub Time Machine ...  14000000   
1   2  [{'id': 107674, 'name': 'The Princess Diaries ...  40000000   
2   3                                                NaN   3300000   
3   4                                                NaN   1200000   
4   5                                                NaN         0   

                                              genres  \
0                     [{'id': 35, 'name': 'Comedy'}]   
1  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
2                      [{'id': 18, 'name': 'Drama'}]   
3  [{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...   
4  [{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...   

                            homepage    imdb_id original_language  \
0                                NaN  tt2637294                en   
1                                NaN  tt0368933                en   
2  http://sonyclassics.com/whiplash

In [4]:
#create subdirectory to save figures

# Data Organization

In [5]:
mkdir models

mkdir: models: File exists


In [6]:
mkdir pics

mkdir: pics: File exists


In [7]:
os.listdir()

['.DS_Store',
 'Dataset',
 'pics',
 'models',
 '.ipynb_checkpoints',
 'capstone2_wrangling.ipynb']

# Data Definition

In [8]:
#column names
df.columns

Index(['id', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'runtime', 'spoken_languages',
       'status', 'tagline', 'title', 'Keywords', 'cast', 'crew', 'revenue'],
      dtype='object')

In [9]:
#data type
df.dtypes

id                         int64
belongs_to_collection     object
budget                     int64
genres                    object
homepage                  object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity               float64
poster_path               object
production_companies      object
production_countries      object
release_date              object
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
Keywords                  object
cast                      object
crew                      object
revenue                    int64
dtype: object

In [10]:
#info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     3000 non-null   int64  
 1   belongs_to_collection  604 non-null    object 
 2   budget                 3000 non-null   int64  
 3   genres                 2993 non-null   object 
 4   homepage               946 non-null    object 
 5   imdb_id                3000 non-null   object 
 6   original_language      3000 non-null   object 
 7   original_title         3000 non-null   object 
 8   overview               2992 non-null   object 
 9   popularity             3000 non-null   float64
 10  poster_path            2999 non-null   object 
 11  production_companies   2844 non-null   object 
 12  production_countries   2945 non-null   object 
 13  release_date           3000 non-null   object 
 14  runtime                2998 non-null   float64
 15  spok

In [11]:
print(df.isna().sum())

id                          0
belongs_to_collection    2396
budget                      0
genres                      7
homepage                 2054
imdb_id                     0
original_language           0
original_title              0
overview                    8
popularity                  0
poster_path                 1
production_companies      156
production_countries       55
release_date                0
runtime                     2
spoken_languages           20
status                      0
tagline                   597
title                       0
Keywords                  276
cast                       13
crew                       16
revenue                     0
dtype: int64


In [12]:
df.nunique()/df.count()

id                       1.000000
belongs_to_collection    0.698675
budget                   0.135333
genres                   0.291346
homepage                 0.994715
imdb_id                  1.000000
original_language        0.012000
original_title           0.991667
overview                 1.000000
popularity               0.999667
poster_path              1.000000
production_companies     0.837904
production_countries     0.108998
release_date             0.799333
runtime                  0.046364
spoken_languages         0.134564
status                   0.000667
tagline                  0.998752
title                    0.989667
Keywords                 0.972100
cast                     0.995983
crew                     1.000000
revenue                  0.950000
dtype: float64

In [13]:
#duplicate check
duplicate_df = df[df.duplicated()]
print(duplicate_df.shape)
# there is no duplicate data

(0, 23)


In [14]:
df.describe()

,id,budget,popularity,runtime,revenue
count,3000.000000,3.000000e+03,3000.000000,2998.000000,3.000000e+03
mean,1500.500000,2.253133e+07,8.463274,107.856571,6.672585e+07
std,866.169729,3.702609e+07,12.104000,22.086434,1.375323e+08
min,1.000000,0.000000e+00,0.000001,0.000000,1.000000e+00
25%,750.750000,0.000000e+00,4.018053,94.000000,2.379808e+06
50%,1500.500000,8.000000e+06,7.374861,104.000000,1.680707e+07
75%,2250.250000,2.900000e+07,10.890983,118.000000,6.891920e+07
max,3000.000000,3.800000e+08,294.337037,338.000000,1.519558e+09


# roadmap
# what need to do with data
1. drop columns:
    id is duplicated to index, will drop it
    belongs_to_collection and homepage columns have over 2/3 data missing, both columns content should be similar as overview or keywords, and it is difficult to fill up over 2/3 data with limited source, so will drop these two columns.
2. fill in numericl NAN data
    only 'runtime' has NAN, and only 2/3000 missing, will fill with average time.data type conversion:
3. drop more columns:
    imdb_id, which is identical id linked to imdb, which is not useful for this case study, will drop it
    overview, the tagline and keyword has contained key info for movie analysis, this is duplicated
4.  'release date' convert to datatime type


# drop columns

In [15]:
df = df.drop(['id','belongs_to_collection','homepage','imdb_id','overview'], axis = 1)

In [16]:
df.iloc[0]

budget                                                           14000000
genres                                     [{'id': 35, 'name': 'Comedy'}]
original_language                                                      en
original_title                                     Hot Tub Time Machine 2
popularity                                                        6.57539
poster_path                              /tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg
production_companies    [{'name': 'Paramount Pictures', 'id': 4}, {'na...
production_countries    [{'iso_3166_1': 'US', 'name': 'United States o...
release_date                                                      2/20/15
runtime                                                                93
spoken_languages                 [{'iso_639_1': 'en', 'name': 'English'}]
status                                                           Released
tagline                 The Laws of Space and Time are About to be Vio...
title                                 

# check and fill numerical NAN

In [17]:
nas=pd.DataFrame(df.isnull().sum().sort_values(ascending=False)/len(df),columns = ['percent'])
pos = nas['percent'] > 0
nas[pos]

,percent
tagline,0.199000
Keywords,0.092000
production_companies,0.052000
production_countries,0.018333
spoken_languages,0.006667
crew,0.005333
cast,0.004333
genres,0.002333
runtime,0.000667
poster_path,0.000333


In [18]:
# fill in zero for null runtime, then all numerical data are ready
df['runtime'] = df['runtime'].fillna(value = 0)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                3000 non-null   int64  
 1   genres                2993 non-null   object 
 2   original_language     3000 non-null   object 
 3   original_title        3000 non-null   object 
 4   popularity            3000 non-null   float64
 5   poster_path           2999 non-null   object 
 6   production_companies  2844 non-null   object 
 7   production_countries  2945 non-null   object 
 8   release_date          3000 non-null   object 
 9   runtime               3000 non-null   float64
 10  spoken_languages      2980 non-null   object 
 11  status                3000 non-null   object 
 12  tagline               2403 non-null   object 
 13  title                 3000 non-null   object 
 14  Keywords              2724 non-null   object 
 15  cast                 

In [20]:
#convert release time to datatime
#leave all others to feature engineering

In [21]:
df.to_csv('/Users/Sue/Documents/study2020/springboard/capstone_project2_movie/Dataset/tmdb-box-office-prediction/0721wragling_t1.csv',index = False)

In [22]:
# define get_dictioinary function to achieve dictionary from lists
def get_dictionary(s):
    try:
        d = eval(s)
    except:
        d = {}
    return d

In [23]:
# clean up 3 columns including keywords
df_train = df
list_process = ['genres','production_companies','production_countries','Keywords']
list_process2 = ['spoken_languages']
for i in list_process:
    df_train[i] = df_train[i].map(lambda x: sorted([d['name'] for d in get_dictionary(x)])).map(lambda x: ','.join(map(str, x)))
    #gi = df_train[i].str.get_dummies(sep=',')
    #df_train = pd.concat([df_train, gi], axis=1, sort=False)
for i in list_process2:
    df_train[i] = df_train[i].map(lambda x: sorted([d['iso_639_1'] for d in get_dictionary(x)])).map(lambda x: ','.join(map(str, x)))
    #gi = df_train[i].str.get_dummies(sep=',')
    #df_train = pd.concat([df_train, gi], axis=1, sort=False)

In [24]:
# clean up column genres
#df_train['genres'] = df_train['genres'].map(lambda x: sorted([d['name'] for d in get_dictionary(x)])).map(lambda x: ','.join(map(str, x)))
#genres_split = df_train['genres'].str.get_dummies(sep=',')
#df_train = pd.concat([df_train, genres_split], axis=1, sort=False)

In [25]:
#df_train.head(5)

# release date convert to datetime datatype

In [26]:
df_train['release_date'].dtype

dtype('O')

In [27]:
df_train['release_date']=pd.to_datetime(df_train['release_date'])

In [28]:
df_train['release_date'][0]

Timestamp('2015-02-20 00:00:00')

In [29]:
df_train.head(5)

,budget,genres,original_language,original_title,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,14000000,Comedy,en,Hot Tub Time Machine 2,6.575393,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,"Metro-Goldwyn-Mayer (MGM),Paramount Pictures,U...",United States of America,2015-02-20,93.0,en,Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"duringcreditsstinger,hot tub,sequel,time travel","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,40000000,"Comedy,Drama,Family,Romance",en,The Princess Diaries 2: Royal Engagement,8.248895,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,Walt Disney Pictures,United States of America,2004-08-06,113.0,en,Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"coronation,duty,falling in love,marriage","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3300000,Drama,en,Whiplash,64.299990,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,"Blumhouse Productions,Bold Films,Right of Way ...",United States of America,2014-10-10,105.0,en,Released,The road to greatness can take you to the edge.,Whiplash,"conservatory,drummer,jazz,jazz band,montage,mu...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,1200000,"Drama,Thriller",hi,Kahaani,3.174936,/aTXRaPrWSinhcmCrcfJK17urp3F.jpg,,India,2012-03-09,122.0,"en,hi",Released,NaN,Kahaani,"bollywood,crime,india,missing husband,mystery,...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,0,"Action,Thriller",ko,마린보이,1.148070,/m22s7zvkVFDU9ir56PiiqIEWFdT.jpg,,South Korea,2009-02-05,118.0,ko,Released,NaN,Marine Boy,,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


In [30]:
df_train.to_csv('/Users/Sue/Documents/study2020/springboard/capstone_project2_movie/Dataset/tmdb-box-office-prediction/0721wragling_train_t2.csv',index = False)